# PyPower

In [1]:
from pypower.runpf import runpf
import pypower

In [2]:
import sys
sys.path.append("../data/case9/")

In [3]:
from case9 import case9

In [4]:
case = case9()

![]("./9BusSystem.jpg")

In [5]:
ppopt = pypower.ppoption.ppoption(VERBOSE=0)

### Initialization

In [6]:
# -*- coding: utf-8 -*-
import time
import helics as h
from math import pi

initstring = "-f 2 --name=mainbroker"
fedinitstring = "--broker=mainbroker --federates=1"
deltat = 0.01

helicsversion = h.helicsGetVersion()

print("Helics version = {}".format(helicsversion))

Helics version = 3.5.2 (2024-04-08)


### Create the broker

In [7]:

print("Creating Broker")
broker = h.helicsCreateBroker("zmq", "", initstring)
print("Created Broker")

print("Checking if Broker is connected")
isconnected = h.helicsBrokerIsConnected(broker)
print("Checked if Broker is connected")

if isconnected == 1:
    print("Broker created and connected")

Creating Broker
Created Broker
Checking if Broker is connected
Checked if Broker is connected
Broker created and connected


### Create the federate info object

In [8]:
# Create Federate Info object that describes the federate properties #
fedinfo = h.helicsCreateFederateInfo()

# Set Federate name #
h.helicsFederateInfoSetCoreName(fedinfo, "PyPowerFederate")

# Set core type from string #
h.helicsFederateInfoSetCoreTypeFromString(fedinfo, "zmq")

# Federate init string #
h.helicsFederateInfoSetCoreInitString(fedinfo, fedinitstring)

# Set the message interval (timedelta) for federate. Note th#
# HELICS minimum message time interval is 1 ns and by default
# it uses a time delta of 1 second. What is provided to the
# setTimedelta routine is a multiplier for the default timedelta.

# Set one second message interval #
h.helicsFederateInfoSetTimeProperty(fedinfo, h.helics_property_time_delta, deltat)

### Create a value federate

In [9]:
# Create value federate #
vfed = h.helicsCreateValueFederate("PyPowerFederate", fedinfo)
print("Value federate created")

# Register the publication #
pub = h.helicsFederateRegisterGlobalTypePublication(vfed, "voltage", "complex", "")
sub = h.helicsFederateRegisterSubscription(vfed, "load", "")
sub.set_default(0.0)

Value federate created


### Enter execution

In [10]:
# Enter execution mode #
h.helicsFederateEnterExecutingMode(vfed)
print("Entering execution mode")

Entering execution mode


### Start simulation

In [11]:
import numpy as np

In [12]:
def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return x, y

In [13]:

granted_time = -1

for request_time in range(1, 300):
    
    while granted_time < request_time:
        granted_time = h.helicsFederateRequestTime(vfed, request_time)
    
    complex_power = h.helicsInputGetComplex(sub)
    Pg, Qg = complex_power.real, complex_power.imag
    print("Active power at time {}: {} ".format(request_time, Pg))
    print("Reactive power at time {}: {} ".format(request_time, Qg))

    case["bus"][4][3:5] = Pg, Qg
    r, s = runpf(case, fname="/dev/null", ppopt=ppopt)
    voltage, angle = r["bus"][4][7:9]
    
    print("Source voltage mag at time {}: {} ".format(request_time, voltage))
    print("Source voltage ang at time {}: {} ".format(request_time, angle))

    h.helicsPublicationPublishComplex(pub, voltage, angle)
    
    print()


h.helicsFederateDisconnect(vfed)
print("Federate disconnected")

while h.helicsBrokerIsConnected(broker):
    time.sleep(1)

h.helicsFederateFree(vfed)
h.helicsCloseLibrary()

print("Broker disconnected")

Active power at time 1: 0.0 
Reactive power at time 1: 0.0 
Source voltage mag at time 1: 1.0031125352761052 
Source voltage ang at time 1: -4.110753858919751 

Active power at time 2: -5.6085993940829045e-146 
Reactive power at time 2: -7.260114379794892e-146 
Source voltage mag at time 2: 1.0031125352761052 
Source voltage ang at time 2: -4.110753858919751 

Active power at time 3: 4.632371195933371 
Reactive power at time 3: 4.046112809180476 
Source voltage mag at time 3: 0.9982575443287671 
Source voltage ang at time 3: -4.419742400565152 

Active power at time 4: 4.632593004603226 
Reactive power at time 4: 4.046368149506048 
Source voltage mag at time 4: 0.998257299996824 
Source voltage ang at time 4: -4.419762054764059 

Active power at time 5: 4.630893230921145 
Reactive power at time 5: 4.0484761661484585 
Source voltage mag at time 5: 0.9982584711931207 
Source voltage ang at time 5: -4.419936820885168 

Active power at time 6: 4.630988219767404 
Reactive power at time 6: 4